#  When Was the Golden Age of Video Games? 
Video games are big business: the global gaming market is projected to be worth more than $300 billion by 2027 according to Mordor Intelligence. With so much money at stake, the major game publishers are hugely incentivized to create the next big hit. But are games getting better, or has the golden age of video games already passed?

In this project, I will explore the top 400 best-selling video games created between 1977 and 2020. I will compare a dataset on game sales with critic and user reviews to determine whether or not video games have improved as the gaming market has grown.

After I created my database from two csv files, it contains two tables:

**game_sales**

| column | type | meaning |
| :- | -: | :-: |
| game       | varchar | Name of the video game
| platform   | varchar | Gaming platform
| publisher  |  varchar | Game publisher
| developer   | varchar | Game developer
| games_sold  | float | Number of copies sold (millions)
| year    | int |  Release year

**reviews**

|column|type|meaning
| --- | --- | --- |
|game|varchar|Name of the video game
|critic_score|float|Critic score according to Metacritic
|user_score|float|User score according to Metacritic

In [1]:
# Import required modules
import sqlite3
import csv
from sqlalchemy import create_engine, inspect, text
import pandas as pd

## 1.The ten best-selling video games
First I creat the game_sales database and I import the two CSV files content into our SQLite database table.
After that I start my analyze by looking at some of the top selling video games of all time.

In [2]:
# Creating a o connection to the database
connection = sqlite3.connect('database.sqlite')

In [3]:
# Creating a cursor object to execute SQL queries 
cursor = connection.cursor()

In [4]:
# Creating game_sales table in the database
cursor.execute('''DROP TABLE game_sales''')
cursor.execute('''CREATE TABLE game_sales (
  game VARCHAR(100) PRIMARY KEY,
  platform VARCHAR(64),
  publisher VARCHAR(64),
  developer VARCHAR(64),
  games_sold NUMERIC(5, 2),
  year INT
);
''')

In [5]:
# Creating reviews table in the database
cursor.execute('''DROP TABLE reviews''')
cursor.execute('''CREATE TABLE reviews (
    game VARCHAR(100) PRIMARY KEY,
    critic_score NUMERIC(4, 2),   
    user_score NUMERIC(4, 2)
);
''')

In [6]:
# Opening the csv file
file1 = open('game_sales.csv')
file2 = open('game_reviews.csv')

In [7]:
# Reading the contents of the csv file
first_row1 = next(csv.reader(file1)) # Skip the first row
content1 = csv.reader(file1)
first_row2 = next(csv.reader(file2)) # Skip the first row
content2 = csv.reader(file2)

In [8]:
# Importing the contents of the file into tabel
cursor.executemany('INSERT INTO game_sales VALUES (?, ?, ?, ?, ?, ?)', content1)
cursor.executemany('INSERT INTO  reviews VALUES (?, ?, ?)', content2)
connection.commit()

In [9]:
# SQL query to retrive 10 rows from the tabel to verify that the data from csv file was inserted into the table
rows1 = cursor.execute('SELECT * FROM game_sales LIMIT 10').fetchall()
rows2 = cursor.execute('SELECT * FROM reviews LIMIT 10').fetchall()

In [10]:
# Display the name and data type of every column of the game_sales table 
content_tabe1 = cursor.execute('PRAGMA table_info (game_sales)').fetchall()
content_tabe1 

[(0, 'game', 'VARCHAR(100)', 0, None, 1),
 (1, 'platform', 'VARCHAR(64)', 0, None, 0),
 (2, 'publisher', 'VARCHAR(64)', 0, None, 0),
 (3, 'developer', 'VARCHAR(64)', 0, None, 0),
 (4, 'games_sold', 'NUMERIC(5, 2)', 0, None, 0),
 (5, 'year', 'INT', 0, None, 0)]

In [11]:
# Display the name and data type of every column of the reviews table 
content_tabe2 = cursor.execute('PRAGMA table_info (reviews)').fetchall()
content_tabe2

[(0, 'game', 'VARCHAR(100)', 0, None, 1),
 (1, 'critic_score', 'NUMERIC(4, 2)', 0, None, 0),
 (2, 'user_score', 'NUMERIC(4, 2)', 0, None, 0)]

In [12]:
# Output to the consol screen from game_sales tabel
game_sales = pd.DataFrame(rows1, columns = [x[1] for x in content_tabe1])
game_sales

,game,platform,publisher,developer,games_sold,year
0,7 Days to Die for PC,PC,The Fun Pimps,The Fun Pimps,4.18,2013
1,ARK: Survival Evolved for PC,PC,Studio Wildcard,Studio Wildcard,4.50,2015
2,Age of Empires II: HD Edition for PC,PC,Microsoft Studios,Hidden Path Entertainment,5.82,2013
3,Animal Crossing: City Folk for Wii,Wii,Nintendo,Nintendo EAD,4.32,2008
4,Animal Crossing: New Horizons for NS,NS,Nintendo,Nintendo,13.41,2020
5,Animal Crossing: New Leaf for 3DS,3DS,Nintendo,Nintendo EAD,12.55,2013
6,Animal Crossing: Wild World for DS,DS,Nintendo,Nintendo EAD,11.75,2005
7,Arma 2: Operation Arrowhead for PC,PC,Meridian4,Bohemia Interactive,4.51,2010
8,Arma III for PC,PC,Bohemia Interactive,Bohemia Interactive,4.00,2013
9,Assassin's Creed II for PS3,PS3,Ubisoft,Ubisoft Montreal,5.57,2009


In [13]:
# Output to the consol screen from reviews tabel
reviews = pd.DataFrame(rows2, columns=[x[1] for x in content_tabe2])
reviews

,game,critic_score,user_score
0,Wii Sports for Wii,7.7,8.0
1,Super Mario Bros. for NES,10.0,8.2
2,Counter-Strike: Global Offensive for PC,8.0,7.5
3,Mario Kart Wii for Wii,8.2,9.1
4,PLAYERUNKNOWN'S BATTLEGROUNDS for PC,8.6,4.7
5,Minecraft for PC,10.0,7.8
6,Wii Sports Resort for Wii,8.0,8.8
7,Pokemon Red / Green / Blue Version for GB,9.4,8.8
8,New Super Mario Bros. for DS,9.1,8.1
9,New Super Mario Bros. Wii for Wii,8.6,9.2


In [14]:
# Select all information for the top ten best-selling games
# Order the results from best-selling game down to tenth best-selling
ts = cursor.execute('''SELECT *
    FROM game_sales
    ORDER BY games_sold DESC
    LIMIT 10;''').fetchall()
df_top_selling = pd.DataFrame(ts, columns = [x[1] for x in content_tabe1])
df_top_selling.sort_values(by = ['year'])

,game,platform,publisher,developer,games_sold,year
1,Super Mario Bros. for NES,NES,Nintendo,Nintendo EAD,40.24,1985
7,Pokemon Red / Green / Blue Version for GB,GB,Nintendo,Game Freak,31.38,1998
0,Wii Sports for Wii,Wii,Nintendo,Nintendo EAD,82.90,2006
8,New Super Mario Bros. for DS,DS,Nintendo,Nintendo EAD,30.80,2006
3,Mario Kart Wii for Wii,Wii,Nintendo,Nintendo EAD,37.32,2008
6,Wii Sports Resort for Wii,Wii,Nintendo,Nintendo EAD,33.13,2009
9,New Super Mario Bros. Wii for Wii,Wii,Nintendo,Nintendo EAD,30.30,2009
5,Minecraft for PC,PC,Mojang,Mojang AB,33.15,2010
2,Counter-Strike: Global Offensive for PC,PC,Valve,Valve Corporation,40.00,2012
4,PLAYERUNKNOWN'S BATTLEGROUNDS for PC,PC,PUBG Corporation,PUBG Corporation,36.60,2017


## 2. Missing review scores
The best-selling video games were released between 1985 to 2017! That's quite a range; I'll have to use data from the reviews table to gain more insight on the best years for video games.

First, it's important to explore the limitations of our database. One big shortcoming is that there is not any reviews data for some of the games on the game_sales table.

In [15]:
# Join games_sales and reviews
# Select a count of the number of games where both critic_score and user_score are null
mrs = cursor.execute('''SELECT COUNT(*)
    FROM game_sales AS g 
    LEFT JOIN reviews AS r
    ON g.game=r.game
    WHERE r.critic_score IS NULL AND r.user_score IS NULL;''').fetchall()
print('The number of games where both critic_score and user_score are null:',  mrs[0][0])

The number of games where both critic_score and user_score are null: 31


## 3. Years that video game critics loved
It looks like a little less than ten percent of the games on the game_sales table don't have any reviews data. That's a small enough percentage that I can continue my exploration.

There are lots of ways to measure the best years for video games! Let's start with what the critics think.

In [16]:
# Select release year and average critic score for each year, rounded and aliased
# Join the game_sales and reviews tables
# Group by release year
# Order the data from highest to lowest avg_critic_score and limit to 10 results

acs = cursor.execute('''SELECT year, ROUND(AVG(critic_score), 2) AS avg_critic_score
    FROM game_sales AS g
    INNER JOIN reviews AS r
    ON g.game=r.game
    GROUP BY year
    ORDER BY avg_critic_score DESC
    LIMIT 10;''').fetchall()
top_critic_years = pd.DataFrame(acs, columns = ['Year','avg_critic_score']).sort_values(by =['Year']).reset_index(drop=True)
print('Top Critics Years Dataset: \n', top_critic_years)

Top Critics Years Dataset: 
    Year  avg_critic_score
0  1982              9.00
1  1990              9.80
2  1992              9.67
3  1993              9.10
4  1995              9.07
5  1998              9.32
6  1999              8.93
7  2002              8.99
8  2004              9.03
9  2020              9.20


## 4. Was 1982 really that great?
The range of great years according to critic reviews goes from 1982 until 2020: I am no closer to finding the golden age of video games!

Some of those avg_critic_score values look like suspiciously round numbers for averages - the value for 1982. Maybe there weren't a lot of video games in our dataset that were released in certain years.

I update the previous query in order to find out whether 1982 really was such a great year for video games.

In [17]:
# Update the previous query so that it only returns years that have more than four reviewed games
acs_four_rev = cursor.execute('''SELECT year, 
    ROUND(AVG(critic_score), 2) AS avg_critic_score,
    COUNT(*) AS num_games
    FROM game_sales AS g
    INNER JOIN reviews AS r
    ON g.game=r.game
    GROUP BY year
    HAVING COUNT(*) > 4
    ORDER BY avg_critic_score DESC
    LIMIT 10;''').fetchall()
top_critic_years_more_than_four_games = pd.DataFrame(acs_four_rev, columns = ['Year','avg_critic_score', 'num_games']).sort_values(by =['Year']).reset_index(drop='True')
print('Top critics years more than four games dataset: \n', top_critic_years_more_than_four_games)

Top critics years more than four games dataset: 
    Year  avg_critic_score  num_games
0  1998              9.32         10
1  1999              8.93         11
2  2001              8.82         13
3  2002              8.99          9
4  2004              9.03         11
5  2008              8.63         20
6  2011              8.76         26
7  2013              8.66         18
8  2016              8.67         13
9  2017              8.62         13


# 5. Years that dropped off the critics' favorites list
The num_games column convinces me that my new list of the critics' top games reflects years that had quite a few well-reviewed games rather than just one or two hits. But which years dropped off the list due to having four or fewer reviewed games? I will identify them.
To get started, first I will creat tables with the results of my previous two queries:

In [18]:
# Create a referenc eengine
engine = create_engine('sqlite:///database.sqlite')
 
# Write records stored in a DataFrame to a SQL database
top_critic_years.to_sql('top_critic_years', if_exists = 'replace', con = engine)
top_critic_years_more_than_four_games.to_sql('top_critic_years_more_than_four_games', if_exists = 'replace', con = engine)

10

In [19]:
# Use an inspector to fetch the list of tables name
tabel_names = inspect(engine).get_table_names()
tabel_names

['game_sales',
 'reviews',
 'top_critic_years',
 'top_critic_years_more_than_four_games',
 'top_user_years_more_than_four_games']

In [20]:
table_content3 =cursor.execute('PRAGMA table_info(top_critic_years)').fetchall()
table_content3 

[(0, 'index', 'BIGINT', 0, None, 0),
 (1, 'Year', 'BIGINT', 0, None, 0),
 (2, 'avg_critic_score', 'FLOAT', 0, None, 0)]

In [21]:
# Display the name and data type of every column of the new tables
table_content3 = pd.read_sql(text('PRAGMA table_info(top_critic_years)'), engine.connect()).set_index('cid')
table_content4 = pd.read_sql(text('PRAGMA table_info(top_critic_years)'), engine.connect()).set_index('cid')
print(table_content3, '\n', table_content4)

                 name    type  notnull dflt_value  pk
cid                                                  
0               index  BIGINT        0       None   0
1                Year  BIGINT        0       None   0
2    avg_critic_score   FLOAT        0       None   0 
                  name    type  notnull dflt_value  pk
cid                                                  
0               index  BIGINT        0       None   0
1                Year  BIGINT        0       None   0
2    avg_critic_score   FLOAT        0       None   0


In [22]:
## Select the year and avg_critic_score for those years that dropped off the list of critic favorites 
## Order the results from highest to lowest avg_critic_score
years_dropped_critics_favorites_list = pd.read_sql_query(text('''SELECT *
    FROM top_critic_years 
    WHERE year NOT IN (SELECT year
    FROM top_critic_years_more_than_four_games)'''), engine.connect()).drop('index', axis = 1)
years_dropped_critics_favorites_list

,Year,avg_critic_score
0,1982,9.00
1,1990,9.80
2,1992,9.67
3,1993,9.10
4,1995,9.07
5,2020,9.20


# 6. Years video game players loved
It looks like the early 1990s might merit consideration as the golden age of video games based on critic_score alone, but we'd need to gather more games and reviews data to do further analysis.

Let's look at the opinions of another important group of people: players! For that I will create a query very similar to the one I used in Task Four, except this one will look at user_score averages by year rather than critic_score averages.

In [23]:
# Select year, an average of user_score, and a count of games released in a given year, aliased and rounded
# Include only years with more than four reviewed games; group data by year
# Order data by avg_user_score, and limit to ten results
user_score_averages = pd.read_sql_query(text('''SELECT year, 
    ROUND(AVG(user_score), 2) AS avg_user_score,
    COUNT(*) AS num_games
    FROM game_sales AS g
    INNER JOIN reviews AS r
    ON g.game=r.game
    GROUP BY year
    HAVING COUNT(*) > 4
    ORDER BY avg_user_score DESC
    LIMIT 10;'''), engine.connect()).sort_values(by =['year']).reset_index(drop='True')
user_score_averages

,year,avg_user_score,num_games
0,2008,4.97,20
1,2009,4.13,20
2,2012,4.27,12
3,2013,4.57,18
4,2014,5.22,22
5,2015,6.64,19
6,2016,6.95,13
7,2017,6.25,13
8,2018,7.03,12
9,2019,5.09,9


# 7. Years that both players and critics loved
I've got a list of the top ten years according to both critic reviews and user reviews. I will check now if are there any years that showed up on both tables. If so, those years would certainly be excellent ones!
I'll also save the results of the top user years query from the previous task into a table.

In [24]:
# Write records stored in a DataFrame to a SQL database
user_score_averages.to_sql('top_user_years_more_than_four_games', if_exists = 'replace', con = engine)
inspect(engine).get_table_names()

['game_sales',
 'reviews',
 'top_critic_years',
 'top_critic_years_more_than_four_games',
 'top_user_years_more_than_four_games']

In [25]:
# Display the name and data type of every column of the new table
tabel_content5 = pd.read_sql_query(text('PRAGMA table_info(top_user_years_more_than_four_games)'),
                                   engine.connect()).set_index('cid')
tabel_content5 

,name,type,notnull,dflt_value,pk
cid,,,,,
0,index,BIGINT,0,None,0
1,year,BIGINT,0,None,0
2,avg_user_score,FLOAT,0,None,0
3,num_games,BIGINT,0,None,0


In [26]:
# Select the year results that appear on both tables
years_that_appear_both_tables = pd.read_sql_query(text('''SELECT u.year
    FROM top_critic_years_more_than_four_games AS c
    INNER JOIN  top_user_years_more_than_four_games AS u
    ON c.year = u.year'''), engine.connect())
years_that_appear_both_tables

,year
0,2008
1,2013
2,2016
3,2017


# 8. Sales in the best video game years
Looks like we've got four years that both users and critics agreed were in the top ten!  We know that critics and players liked these years, but what about video game makers? Were sales good? Let's find out.

This time, I won't save the results from the previous task in a new table. Instead, I'll use the query from the previous task as a subquery in this one.

In [27]:
# Select year and sum of games_sold, aliased as total_games_sold; order results by total_games_sold descending
# Filter game_sales based on whether each year is in the list returned in the previous task
sales_best_video_game_years = pd.read_sql_query(text(''' SELECT year, 
    SUM(games_sold) AS total_games_sold
    FROM game_sales
    WHERE year IN (SELECT u.year
    FROM top_critic_years_more_than_four_games AS c
    INNER JOIN  top_user_years_more_than_four_games AS u
    ON c.year = u.year)
    GROUP BY year
    ORDER BY total_games_sold DESC;'''), engine.connect()).sort_values('year')
sales_best_video_game_years

,year,total_games_sold
1,2008,175.07
0,2013,177.23
3,2016,105.01
2,2017,174.08


In [28]:
# closing the database connection and the csv files
connection.close()
file1.close()
file2.close()